In [1]:
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # Accuracy metrics 


In [2]:
df = pd.read_csv('Dataset/csv/v1_coords.csv')

In [3]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1234)

In [4]:
y_test

44            jab
7             jab
254    uppercut_r
223    uppercut_l
90       straight
91       straight
210    uppercut_l
162        hook_r
335          idle
283    uppercut_r
43            jab
110        hook_l
287    uppercut_r
179        hook_r
59       straight
133        hook_l
191        hook_r
169        hook_r
225    uppercut_l
54       straight
104      straight
157        hook_r
181        hook_r
193        hook_r
156        hook_r
200        hook_r
343          idle
274    uppercut_r
207    uppercut_l
248    uppercut_l
92       straight
22            jab
108        hook_l
347          idle
295    uppercut_r
70       straight
168        hook_r
205    uppercut_l
297    uppercut_r
37            jab
216    uppercut_l
64       straight
286    uppercut_r
13            jab
101      straight
71       straight
73       straight
134        hook_l
270    uppercut_r
125        hook_l
66       straight
194        hook_r
230    uppercut_l
217    uppercut_l
Name: class, dtype: object

In [5]:
nan_values = df.isna().sum()
print(nan_values)

class    0
x1       0
y1       0
z1       0
v1       0
x2       0
y2       0
z2       0
v2       0
x3       0
y3       0
z3       0
v3       0
x4       0
y4       0
z4       0
v4       0
x5       0
y5       0
z5       0
v5       0
x6       0
y6       0
z6       0
v6       0
x7       0
y7       0
z7       0
v7       0
x8       0
y8       0
z8       0
v8       0
x9       0
y9       0
z9       0
v9       0
x10      0
y10      0
z10      0
v10      0
dtype: int64


# Pipeline of the classfication

In [6]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.svm import *

In [7]:
model = make_pipeline(StandardScaler(), SVC(probability=True)).fit(X_train, y_train)
model.predict(X_test)

array(['jab', 'jab', 'uppercut_r', 'uppercut_l', 'straight', 'hook_r',
       'uppercut_l', 'hook_r', 'idle', 'uppercut_r', 'jab', 'hook_l',
       'uppercut_r', 'hook_r', 'straight', 'hook_l', 'hook_r', 'hook_r',
       'uppercut_l', 'straight', 'straight', 'hook_r', 'hook_r', 'hook_r',
       'hook_r', 'hook_r', 'idle', 'uppercut_r', 'uppercut_l',
       'uppercut_l', 'straight', 'jab', 'hook_l', 'idle', 'uppercut_r',
       'straight', 'hook_r', 'uppercut_l', 'uppercut_r', 'jab',
       'uppercut_l', 'straight', 'uppercut_r', 'jab', 'hook_r',
       'straight', 'straight', 'hook_l', 'uppercut_r', 'hook_l',
       'straight', 'hook_r', 'uppercut_l', 'uppercut_l'], dtype=object)

In [8]:
version = 1
with open(f'../Model/v{version}_model.pkl', 'wb') as f:
    pickle.dump(model, f)